模型从数据中学习：模型的训练过程

In [137]:
import numpy as np

In [138]:
# 损失函数
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [139]:
# 一个计算的例子
# 设“2”为正确解
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
# 例1：“2”的概率最高的情况（0.6）
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
mean_squared_error(np.array(y), np.array(t))

0.09750000000000003

In [140]:
# 例2：“7”的概率最高的情况（0.6）
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
mean_squared_error(np.array(y), np.array(t))

0.5975

交叉熵损失函数

$E=-\sum_k{t_k log y_k}$

$y_k$是神经网络的输出，$t_k&是正确解标签。并且，$t_k$ 中只有正确解标签的索引为1，其他均为0（one-hot表示）。

In [141]:
# 交叉熵损失函数
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

In [142]:
# 计算实例
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [143]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

实际上，在学习的过程中，应当把所有训练数据的损失函数都作为目标进行学习。

因此，损失函数可以进一步写为：$E=-\frac{1}{N}\sum_n\sum_k t_{nk}logy_{nk}$

在实际的计算过程中，我们是从训练数据中选择一批去计算损失函数的值。

神经网络的学习也是从训练数据中选出一批数据（称为mini-batch,小批量），然后对每个mini-batch进行学习。

In [144]:
# 如何设定批量？

import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
print(x_train.shape) # (60000, 784)
print(t_train.shape) # (60000, 10)

(60000, 784)
(60000, 10)


In [145]:
# 从所有数据中随机选取10个数据
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [146]:
# 实现batch版的交叉熵
# one-hot版本
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size


In [147]:
# 数字标签版本
def cross_entropy_error_new_label(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [148]:
batch_size = 10
print(np.arange(batch_size))

[0 1 2 3 4 5 6 7 8 9]


In [149]:
t = [2,7,8,4,5,3,2,4,1,8]
y = np.random.uniform(0,1,(10,10))
l = [np.arange(batch_size),np.array(t)]
print(y)
print(l)
print(y[np.arange(batch_size),np.array(t)])

[[0.62462369 0.99127311 0.58472238 0.42636286 0.80585446 0.72039222
  0.36890134 0.00203693 0.6545725  0.51591287]
 [0.82720597 0.20737656 0.25668042 0.14391568 0.23246991 0.87753964
  0.9407443  0.92570053 0.07383895 0.01174901]
 [0.34846777 0.42404329 0.35524991 0.3393804  0.2997245  0.25953985
  0.02977389 0.81367863 0.44669289 0.88051307]
 [0.04953133 0.34553479 0.85132736 0.73755216 0.715663   0.39167197
  0.58783915 0.75015796 0.09112103 0.62883957]
 [0.83206471 0.41865225 0.09756433 0.82680445 0.74473453 0.23053902
  0.0495651  0.1187685  0.74352963 0.74681066]
 [0.558113   0.69933344 0.95408351 0.64389968 0.03163256 0.28978572
  0.77521213 0.14535819 0.21892047 0.94270915]
 [0.5330108  0.0249642  0.09286895 0.47731829 0.67957642 0.1448772
  0.8760873  0.25298544 0.27662369 0.28476743]
 [0.76568195 0.53752116 0.38052437 0.71966969 0.52268852 0.97669221
  0.98841442 0.69979487 0.75150102 0.46601202]
 [0.32089282 0.95453238 0.85501764 0.85916208 0.46060409 0.02977806
  0.58148816 

为什么需要损失函数？

在进行神经网络的学习时，不能将识别精度作为指标。因为如果以识别精度为指标，则参数的导数在绝大多数地方都会变为0。

使用阶跃函数作为激活函数存在同样的问题。

In [150]:
# 数值微分
def numerical_diff(f, x):
    h = 1e-4 # 0.0001
    return (f(x+h) - f(x-h)) / (2*h)

In [151]:
# 例子
def function_1(x):
    return 0.01*x**2 + 0.1*x

numerical_diff(function_1, 5) # 数值解为0.2

0.1999999999990898

In [152]:
# 求偏导的例子
def function_2(x):
    return x[0]**2 + x[1]**2
    # 或者return np.sum(x**2)

In [153]:
def function_tmp1(x0):
    return x0*x0 + 4.0**2.0
numerical_diff(function_tmp1, 3.0)

6.00000000000378

偏导数需要将多个变量中的某一个变量定为目标变量，并将其他变量固定为某个值。

由全部变量的偏导数汇总而成的向量称为梯度（gradient）。

In [154]:
# 求梯度的函数
# 注意：该函数只对一维的权重有效
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) # 生成和x形状相同的数组
    for idx in range(x.size):
        tmp_val = x[idx]
        # f(x+h)的计算
        x[idx] = tmp_val + h
        fxh1 = f(x)
        # f(x-h)的计算
        x[idx] = tmp_val - h
        fxh2 = f(x)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 还原值
    return grad

神经网络的优化过程中，所采用的方法为梯度（下降）法。

In [155]:
# 梯度下降法的简单呈现
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x

In [156]:
# 利用一个普通的函数进行实验
def function_2(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100)

array([-6.11110793e-10,  8.14814391e-10])

In [8]:
# 对一个简单的神经网络求梯度
import sys, os
sys.path.append(os.pardir)
import numpy as np
def numerical_gradient(f, x):
    # 这是对多维权重也有效的数值微分函数
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 値を元に戻す
        it.iternext()   
        
    return grad
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # 溢出对策
    sum_exp_a = np.sum(exp_a,axis=-1, keepdims=True)
    y = exp_a / sum_exp_a
    return y

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # 用高斯分布进行初始化
    def predict(self, x):
        return np.dot(x, self.W)
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss

In [158]:
net = simpleNet()
print(net.W)
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)
np.argmax(p) # 最大值的索引
t = np.array([0, 0, 1]) # 正确解标签
net.loss(x, t)

[[ 1.11618769  0.10679615  1.39532972]
 [-1.46325012 -1.1249419   0.63860369]]
[-0.64721249 -0.94837002  1.41194115]


0.2004500594137805

In [159]:
# 求梯度
def f(W):
    return net.loss(x, t)

dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.06263509  0.04634754 -0.10898263]
 [ 0.09395264  0.06952131 -0.16347395]]


In [160]:
x = np.random.randn(2,3)
print(x)
it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
# 这是用来输出数组索引的迭代对象
# 其中readwrite代表可以对数组对象进行修改
while not it.finished:
    idx = it.multi_index
    print(idx)
    it.iternext()

[[ 0.9575853  -0.05949021  0.32177548]
 [ 0.72889458 -1.03723657  0.06966219]]
(0, 0)
(0, 1)
(0, 2)
(1, 0)
(1, 1)
(1, 2)
